###Competitive ligand binding

Using the ITC model found here:

http://www.sciencedirect.com/science/article/pii/S0003269799944020


Scheme is for competitive binding of $A$ and $B$ to protein $P$:

$$A + P \rightleftharpoons PA$$
$$B + P \rightleftharpoons PB$$

To describe this, we use the following equilibrium constants:

$$K_{A} = \frac{[PA]}{[P]_{free}[A]_{free}}$$

$$K_{B} = \frac{[PB]}{[P]_{free}[B]_{free}}$$


### Determine the relative populations of species in solution

We can only manipulate $[P]_{total}$, $[A]_{total}$ and $[B]_{total}$ experimentally, so our first goal is to determine the concentrations of species such as $[PA]$ which we cannot manipulate or directly observe.  Start by writing concentrations as mole fractions:

$$x_{P} = \frac{[P]_{free}}{[P]_{total}}$$

$$x_{PA} = \frac{[PA]}{[P]_{total}}$$

$$x_{PB} = \frac{[PB]}{[P]_{total}}$$

$$x_{P} + x_{PA} + x_{PB} = 1$$

A root of the binding polynomial has been found that describes $x_{P}$ only in terms of $K_{A}$, $K_{B}$, $[A]_{total}$, $[B]_{total}$ and $[P]_{total}$.  Start with some convenient definitions:

$$c_{A} = K_{A}[P]_{total}$$

$$c_{B} = K_{B}[P]_{total}$$

$$r_{A} = \frac{[A]_{total}}{P_{total}}$$

$$r_{B} = \frac{[B]_{total}}{P_{total}}$$

The value of $x_{P}$ is given by:

$$\alpha = \frac{1}{c_{A}} + \frac{1}{c_{B}} + r_{A} + r_{B} - 1$$

$$\beta = \frac{r_{A}-1}{c_{B}} + \frac{r_{B} - 1}{c_{A}} + \frac{1}{c_{A}c_{B}}$$

$$\gamma = -\frac{1}{c_{A}c_{B}}$$

$$\theta = arccos \Big ( \frac{-2\alpha^{3} + 9\alpha \beta -27\gamma}{2\sqrt{(\alpha^2 - 3 \beta)^3}} \Big) $$

$$x_{P} = \frac{2\sqrt{\alpha^2 - 3 \beta}\ cos(\theta/3) - \alpha}{3}$$

Once this is known $x_{PA}$ and $x_{PB}$ are uniquely determined by:

$$x_{PA} = \frac{r_{A} x_{P}}{1/C_{A} + x_{P}}$$

$$x_{PB} = \frac{r_{B} x_{P}}{1/C_{B} + x_{P}}$$



###Relate to heats and enthalpies

In the paper, they describe the change in heat for each shot $i$ ($\Delta Q_{i}$) as:

$$\Delta Q_{i} = V_{0}P_{total}(\Delta H_{A}(x_{PA,i} - f_{i}x_{PA,i-1}) + \Delta H_{B}(x_{PB,i} - f_{i}x_{PB,i-1})) + q_{dilution},$$

where $V_{0}$ is the volume of the cell, $\Delta H_{A}$ is the enthalpy for binding ligand $A$, $\Delta H_{B}$ is the enthalpy for binding ligand $B$. $f_{i}$ is the dilution factor for each injection: 

$$f_{i} = exp(-V_{i}/V_{0}),$$

where $V_{0}$ is the volume of the cell and $V_{i}$ is the volume of the $i$-th injection.

I've implemented a modified version of this.  My code calculates $x_{PA,i}$ and friends for the entire titration, correcting for dilution.  This means the $f_{i}$ term is superfluous.  Thus, heats are related by:

$$\Delta Q_{i} = V_{0}P_{total,i}(\Delta H_{A}(x_{PA,i} - x_{PA,i-1}) + \Delta H_{B}(x_{PB,i} - x_{PB,i-1})) + q_{dilution}.$$

Note that $V_{0}$ is held constant (it is the cell volume) as only that volume is detected, not the neck of the cell.


In [13]:
%matplotlib inline

# libraries for making pretty sliders and interactive graphs
from IPython.html import widgets
from IPython.html.widgets import interactive
from IPython.display import display

In [14]:
import copy
import numpy as np
import scipy.optimize as optimize
from matplotlib import pyplot as plt

class ITCModel:
    
    def __init__(self):
        pass
    
    def dQ(self):
        pass
    
    @property
    def mole_ratio(self):
        
        return self._A_total[1:]/self._P_total[1:]

class SingleSite(ITCModel):
    
    def __init__(self,
                 P_cell=100e-6,P_syringe=0.0,
                 A_cell=0.0,   A_syringe=1000e-6,
                 cell_volume=300.0,
                 shot_volumes=[2.5 for i in range(30)]):
                 
        """
        P_cell: protein concentration in cell in M
        P_syringe: protein concentration in syringe in M
        A_cell: ligand A concentration cell in M
        A_syringe: ligand A concentration syringe in M
        cell_volume: cell volume, in uL
        shot_volumes: list of shot volumes, in uL. 
        shot_start: first shot to use in fit
        """
        
        self._P_cell = P_cell
        self._P_syringe = P_syringe
        
        self._A_cell = A_cell
        self._A_syringe = A_syringe
            
        self._cell_volume = cell_volume
        self._shot_volumes = np.array(shot_volumes)
        
        self._determine_titration_conc()

    
    def _determine_titration_conc(self):
        """
        Determine the total concentrations of P and A in tn the cell given 
        a set of titration shots and initial concentrations of P and A in
        the cell and syringe.
        """
        
        self._volume  = np.zeros(len(self._shot_volumes)+1)
        self._P_total = np.zeros(len(self._shot_volumes)+1)
        self._A_total = np.zeros(len(self._shot_volumes)+1)
        
        self._volume[0]  = self._cell_volume
        self._P_total[0] = self._P_cell
        self._A_total[0] = self._A_cell
        
        for i in range(len(self._shot_volumes)):
            
            self._volume[i+1] = self._volume[i] + self._shot_volumes[i]
            
            dilution = self._volume[i]/self._volume[i+1]
            added = self._shot_volumes[i]/self._volume[i+1]
            
            self._P_total[i+1] = self._P_total[i]*dilution + self._P_syringe*added
            self._A_total[i+1] = self._A_total[i]*dilution + self._A_syringe*added
            
    def dQ(self,KA,dHA,fx_comp=1.0):
        """
        Calculate the heats that would be observed across shots for a given set of enthalpies
        and binding constants for each reaction.
        """
        
        # ----- Determine mole fractions -----
        P = self._P_total*fx_comp
        b = P + self._A_total + 1/KA
        PA = (b - np.sqrt((b)**2 - 4*P*self._A_total))/2
    
        self._x_pa = PA/P
        self._x_p = 1 - self._x_pa
        
        # ---- Relate mole fractions to heat -----
        A = dHA*(self._x_pa[1:] - self._x_pa[:-1])
    
        to_return = self._cell_volume*(self._P_total[1:]*fx_comp)*A
        
        return to_return

class SingleSiteCompetitor(ITCModel):
    
    def __init__(self,            
                 P_cell=100e-6,P_syringe=0.0,
                 A_cell=0.0,   A_syringe=1000e-6,
                 B_cell=200e-6,B_syringe=0.0,
                 cell_volume=300.0,
                 shot_volumes=[2.5 for i in range(30)]):
                 
        """
        P_cell: protein concentration in cell in M
        P_syringe: protein concentration in syringe in M
        A_cell: ligand A concentration cell in M
        A_syringe: ligand A concentration syringe in M
        B_cell: ligand B concentration cell in M
        B_syringe: ligand B concentration syringe in M
        cell_volume: cell volume, in uL
        shot_volumes: list of shot volumes, in uL. 
        shot_start: first shot to use in fit
        """
        
        self._P_cell = P_cell
        self._P_syringe = P_syringe
         
        self._A_cell = A_cell
        self._A_syringe = A_syringe
        
        self._B_cell = B_cell
        self._B_syringe = B_syringe
        
        self._cell_volume = cell_volume
        self._shot_volumes = np.array(shot_volumes)
        
        self._determine_titration_conc()
                 
                
    def _determine_titration_conc(self):
        """
        Determine the total concentrations of P, A, and B in the cell given 
        a set of titration shots and initial concentrations of P, A and B in
        the cell and syringe.
        """
        
        self._volume  = np.zeros(len(self._shot_volumes)+1)
        self._P_total = np.zeros(len(self._shot_volumes)+1)
        self._A_total = np.zeros(len(self._shot_volumes)+1)
        self._B_total = np.zeros(len(self._shot_volumes)+1)
        
        self._volume[0]  = self._cell_volume
        self._P_total[0] = self._P_cell
        self._A_total[0] = self._A_cell
        self._B_total[0] = self._B_cell
        
        for i in range(len(self._shot_volumes)):
            
            self._volume[i+1] = self._volume[i] + self._shot_volumes[i]
            
            dilution = self._volume[i]/self._volume[i+1]
            added = self._shot_volumes[i]/self._volume[i+1]
            
            self._P_total[i+1] = self._P_total[i]*dilution + self._P_syringe*added
            self._A_total[i+1] = self._A_total[i]*dilution + self._A_syringe*added
            self._B_total[i+1] = self._B_total[i]*dilution + self._B_syringe*added


    def dQ(self,KA,KB,dHA,dHB,fx_comp=1.0):
        """
        Calculate the heats that would be observed across shots for a given set of enthalpies
        and binding constants for each reaction.
        """
    
        # ----- Determine mole fractions -----
        P = self._P_total*fx_comp
    
        c_a = KA*P
        c_b = KB*P
        r_a = self._A_total/(P)
        r_b = self._B_total/(P)
        
        alpha = 1/c_a + 1/c_b + r_a + r_b - 1
        beta = (r_a - 1)/c_b + (r_b - 1)/c_a + 1/(c_a*c_b)
        gamma = -1/(c_a*c_b)
        theta = np.arccos((-2*alpha**3 + 9*alpha*beta - 27*gamma)/(2*np.sqrt((alpha**2 - 3*beta)**3)))
    
        self._x_p = (2*np.sqrt(alpha**2 - 3*beta) * np.cos(theta/3) - alpha)/3
        self._x_pa = r_a*self._x_p/(1/c_a + self._x_p)
        self._x_pb = r_b*self._x_p/(1/c_b + self._x_p)
    
        # ---- Relate mole fractions to heat -----
        A = dHA*(self._x_pa[1:] - self._x_pa[:-1])
        B = dHB*(self._x_pb[1:] - self._x_pb[:-1])
        
        to_return = self._cell_volume*P*(A + B)
        
        return to_return
        

In [15]:
class ITCExperiment:
    """
    Class that holds an experimental ITC measurement and a model that describes it. 
    """

    def __init__(self,dh_file,model,shot_start=1,**model_kwargs):
        """
        dh_file: integrated heats file written out by origin software.
        model: name of ITCModel subclass to use for modeling
        shot_start: what shot to use as the first real point.  Shots 
                    start at 0, so default=1 discards first point.
        **model_kwargs: any keyword arguments to pass to the model.  Any 
                        keywords passed here will override whatever is 
                        stored in the dh_file. 
        """
        
        self._dh_file = dh_file
        self._shot_start = shot_start

        # Load in heats
        self._read_heats_file()

        # Initialize model using information read from heats file
        self._model = model(P_cell=self.stationary_cell_conc,
                            A_syringe=self.titrant_syringe_conc,
                            cell_volume=self.cell_volume,
                            shot_volumes=self._shots,**model_kwargs)
        
    def _read_heats_file(self):
        """
        Read the heats file written out by the MicroCal/Origin ITC analysis
        package. 
        """
        
        
        f = open(self._dh_file,'r')
        lines = f.readlines()
        f.close()
        
        meta = lines[2].split(",")
        
        self.temperature = float(meta[0])
        self.stationary_cell_conc = float(meta[1])*1e-3
        self.titrant_syringe_conc = float(meta[2])*1e-3
        self.cell_volume = float(meta[3])*1e3
        
        shots = []
        heats = []
        for l in lines[5:]:
            col = l.split(",")
            shots.append(float(col[0]))
            heats.append(float(col[1]))
            
        self._shots = np.array(shots)
        self._heats = np.array(heats)
        
    def model_dq(self,**params):
        """
        Return heats calculated by the model with parameters defined in params
        dictionary.
        """
        return self._model.dQ(**params)[self._shot_start:]
        
    @property
    def heats(self):
        """
        Return experimental heats.
        """
        return self._heats[self._shot_start:]
    
    @property
    def mole_ratio(self):
        """
        Return the mole ratio of titrant to stationary.
        """
        return self._model.mole_ratio[self._shot_start:]
        

        
    

In [20]:
class GlobalFit:
    """
    Class for regressing some binding model against an arbitrary number of 
    ITC experiments.
    """

    def __init__(self,param_guesses):
        """
        Set up the main binding model to fit.
        param_guesses: a dictionary of all parameters to be fit.  keys are parameter
                       names.  values are parameter guesses. 
        """

        # Dictionary with all parameters to be fit as keys and initial guesses
        # for their values.
        self._param_guesses = copy.copy(param_guesses)

        # maps for converting between parameter name and index of parameter in 
        # the np.array used for the fitting.
        self._index_to_name = list(self._param_guesses.keys())
        self._index_to_name.sort()
        self._name_to_index = dict([(a,i)
                                    for i,a in enumerate(self._index_to_name)])

        # _fit_param_array holds the parameter values that will be fit by 
        # nonlinear regression
        self._fit_param_array = np.zeros(len(self._param_guesses),dtype=float)
        for a in self._name_to_index.keys():
            self._fit_param_array[self._name_to_index[a]] = self._param_guesses[a]

        # List of experiments and parameter mapping
        self._experiment_list = []
        self._arg_map_list = []
        self._exp_weights = []

    def add_experiment(self,experiment,arg_map,weight=1.0):
        """
        experiment: an initialized ITCExperiment instance
        arg_map: dictionary mapping global parameter names to parameters for individual
                 experiment model parameters.  
        weight: how much to weight this experiment in the regression relative to other
                experiments.  Values <1.0 weight this experiment less than others; 
                values >1.0 weight this more than others.
        """

        self._experiment_list.append(experiment)
        self._arg_map_list.append(copy.copy(arg_map))
        self._exp_weights.append(weight)

        # Make sure that the global parameter values in arg_map are actually found in
        # the set of global parameters
        for a in self._arg_map_list[-1].keys():
            try:
                self._name_to_index[a]
            except KeyError:
                err = "parameter {} not part of fit\n".format(a)
                raise ValueError(err)

    def _residuals(self,param):
        """
        Determine the residuals between the global model and all experiments.
        """
        
        all_residuals = []
        
        # Calculate residuals for each experiment
        for i, e in enumerate(self._experiment_list):

            # This uses the names of the total model parameters as keys to 
            # map to the appropriate parameter values being fit to the 
            # names of those parameters in the local experiment model.
            model_param_kwarg = {}
            for a in self._arg_map_list[i].keys():
                dq_model_param_name = self._arg_map_list[i][a]
                param_value = param[self._name_to_index[a]]
                model_param_kwarg[dq_model_param_name] = param_value

            # calculate the current value of the model
            calc = e.model_dq(**model_param_kwarg)
            
            # add to the total set of residuals
            all_residuals.extend(self._exp_weights[i]*(e.heats - calc))

        return np.array(all_residuals)

    
    def fit(self):
        """
        Perform a global fit using nonlinear regression.
        """
        
        # Do the actual fit
        fit_param, covariance, info_dict, message, error = optimize.leastsq(self._residuals,
                                                                            x0=self._fit_param_array,
                                                                            full_output=True)
        # Store the result
        self._fit_param = fit_param
    
    
    def plot(self,color_list=None):
        """
        Plot the experimental data and fit results.
        """
        
        if color_list == None:
            N = len(self._experiment_list)
            color_list = [plt.cm.brg(i/N) for i in range(N)]
        
        if len(color_list) < len(self._experiment_list):
            err = "Number of specified colors is less than number of experiments.\n"
            raise ValueError(err)
        
        for i, e in enumerate(self._experiment_list):
            
            model_param_kwarg = {}
            for a in self._arg_map_list[i].keys():
                dq_param_name = self._arg_map_list[i][a]
                param_value = self.fit_param[a]
                model_param_kwarg[dq_param_name] = param_value
            
            plt.plot(e.mole_ratio,e.heats,"o",color=color_list[i])
            plt.plot(e.mole_ratio,e.model_dq(**model_param_kwarg),color=color_list[i],linewidth=1.5)
        
    
    @property
    def fit_param(self):
        """
        Return the fit results as a dictionary that keys parameter name to fit value.
        """
        
        out_dict = {}
        for i, a in enumerate(self._index_to_name):
            out_dict[a] = self._fit_param[i]
        
        return out_dict
        
    
    
    

        